In [2]:
import sys
sys.path.append("..")

In [8]:
from text_vae import Hyper, TextVae
import bytelevel

In [4]:
from keras.preprocessing.sequence import pad_sequences

In [9]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np

In [27]:
train = fetch_20newsgroups(subset='train')
x_train = bytelevel.encode(train['data'])
x_train = pad_sequences(x_train, 50)
x_train_one_hot = bytelevel.onehot(x_train)

test = fetch_20newsgroups(subset='test')
x_test = bytelevel.encode(test['data'])
x_test = pad_sequences(x_test, 50)
x_test_one_hot = bytelevel.onehot(x_test)

In [28]:
h = Hyper(vocab_size=256, max_length=50)
model = TextVae(h)

shapes
(?, ?, ?)
<dtype: 'float32'>
(?, 50, 256)


In [29]:
model.fit(x=x_train, y=x_train_one_hot, epochs=2, batch_size=10, validation_data=(x_test, x_test_one_hot))

Train on 11314 samples, validate on 7532 samples
Epoch 1/2
11314/11314 [==============================] - 965s - loss: 0.8454 - val_loss: 0.8274
Epoch 2/2
11314/11314 [==============================] - 954s - loss: 0.8620 - val_loss: 0.8763


In [37]:
model.fit(x=x_train, y=x_train_one_hot, epochs=2, batch_size=10, validation_data=(x_test, x_test_one_hot))

Train on 11314 samples, validate on 7532 samples
Epoch 1/2
11314/11314 [==============================] - 961s - loss: 0.8699 - val_loss: 0.8699
Epoch 2/2
11314/11314 [==============================] - 954s - loss: 0.8669 - val_loss: 0.8691


In [40]:
model.fit(x=x_train, y=x_train_one_hot, epochs=10, batch_size=10, validation_data=(x_test, x_test_one_hot))

Train on 11314 samples, validate on 7532 samples
Epoch 1/10
11314/11314 [==============================] - 963s - loss: 0.8684 - val_loss: 0.8716
Epoch 2/10
11314/11314 [==============================] - 954s - loss: 0.8663 - val_loss: 0.8696
Epoch 3/10
11314/11314 [==============================] - 955s - loss: 0.8666 - val_loss: 0.8741
Epoch 4/10
11314/11314 [==============================] - 955s - loss: 0.8676 - val_loss: 0.8710
Epoch 5/10
11314/11314 [==============================] - 955s - loss: 0.8675 - val_loss: 0.8707
Epoch 6/10
11314/11314 [==============================] - 956s - loss: 0.8657 - val_loss: 0.8660
Epoch 7/10
11314/11314 [==============================] - 956s - loss: 0.8668 - val_loss: 0.8775
Epoch 8/10
11314/11314 [==============================] - 956s - loss: 0.8666 - val_loss: 0.8788
Epoch 9/10
11314/11314 [==============================] - 956s - loss: 0.8677 - val_loss: 0.8698
Epoch 10/10
11314/11314 [==============================] - 957s - loss: 0.8681

In [30]:
x_test[:3]

array([[ 32, 115, 117, 109, 109, 101, 114,  32, 105, 115,  32, 116, 104,
        101,  32,  98, 101, 115, 116,  32, 116, 105, 109, 101,  32, 116,
        111,  32,  98, 117, 121,  46,  10,  10,   9,   9,   9,  78, 101,
        105, 108,  32,  71,  97, 110, 100, 108, 101, 114,  10],
       [ 76,  76,  69,  82,  32,  47,  47,  32,  49,  54,  50,  48,  51,
         32,  87,  79,  79,  68,  83,  32,  47,  47,  32,  77,  85,  83,
         75,  69,  71,  79,  44,  32,  87,  73,  83,  46,  32,  53,  51,
         49,  53,  48,  32,  47,  47,  32,  85,  83,  65,  10],
       [104, 101,  32, 110, 111, 110, 101, 120, 105, 115, 116, 101, 110,
         99, 101,  32, 111, 102,  32,  71, 111, 100,  63,  10,  10,  73,
        110,  32,  97,  32, 119, 111, 114, 100,  44,  32, 121, 101, 115,
         46,  10,  10,  10, 109,  97, 116, 104, 101, 119,  10]],
      dtype=int32)

In [41]:
encode = model.encode(x_test[:3, :50])
decode = model.generate(encode)

In [42]:
bytelevel.prediction2str(decode)

['                                             \n\n\n\n\n',
 '                                             \n\n\n\n\n',
 '                                             \n\n\n\n\n']

In [38]:
b.argmax(axis=-1)

array([16,  6,  6])